In [45]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import scipy.sparse 
import scipy.sparse.linalg

In [46]:
# Parameters : γ = 1,φ = 1,χ = 1,β = 0.99,ρ = 0.99 
# ν varies, simply add diffrent iterations i guess? ν = {0.25, 0.5, 0.75, 1}
# NOTE : I am aware that I have to use the values function to loop over, 
# but I have limited time so will just do this manually for now 

gamma = 1
phi = 1
chi = 1
beta = 0.99
rho = 0.99
T = 100
nu = 0.25



Next we set up the matrices in the market clearing block, which we use to construct  ∂𝐇/∂𝐘 .

In [47]:
# define sparse identity, above-diagonal sparse matrix, below-diagonal sparse matrix, and zero matrix

I = sp.sparse.eye(T)
Ip1 = sp.sparse.diags([np.ones(T-1)], [1], (T, T))
Im1 = sp.sparse.diags([np.ones(T-1)], [-1], (T, T))
Z = sp.sparse.csr_matrix((T, T))
A = sp.sparse.csr_matrix((T, T)) #unsure how to define A 


# market clearing block: goods market clearing

Phigmc = I
Phigmy = -I
Phigmx = Z
Phigmwp = Z
Phigmmp = Z
Phigmq = Z

# market clearing block: Euler equation
Phieulc = nu * I - nu * Ip1
Phieuli = Z
Phieuly = Z
Phieulrk = nu * I - gamma*I 
Phieulwp = Z

# combine matrix blocks in a single sparse matrix with the following structure:
# [Phigmc Phigmi Phigmy Phigmrk Phigmwp]
# [Phieulc Phieuli Phieuly Phieulrk Phieulwp]
dHdY = sp.sparse.bmat([[Phigmc, Phigmy, Phigmx, Phigmwp, Phigmmp],
                       [Phieulc, Phieuli, Phieuly, Phieulrk, Phieulwp]])

assert dHdY.shape == (2*T, 5*T)

Next we construct  ∂𝐘/∂𝐔  and  ∂𝐘/∂𝐙  using the blocks. Start with the firm block. Also adding the bond market 

In [49]:
# firm block matrices: output
Phiyn = I
Phiym = Z
Phiyp = Z

# firm block matrices: labor demand
Phiwpm = Z
Phiwpp = Z
Phiwpn = Z

# household block matrix: labor supply
Phica = 1/gamma * Phiwpm
Phick = (1/A) * phi*I - (1/A)*Phiwpm
Phicn = Z


# there is a Phix() function that goes in to the bond makrket equations but I don't know how to calculate c for it 
#bond market 

Phiqm =nu*I*Phica - (nu-gamma)*I*Phigmx
Phiqn = nu*I*Phicn - (nu-gamma)*I*Phigmn

# combine firm matrices in the following format:
# [Phiyk Phiyn]
# [Phirkk Phirkn]
# [Phiwpk Phiwpn]
dYFdU = sp.sparse.bmat([[Phiym, Phiyn],
                        [Phirkk, Phirkn],
                        [Phiwpm, Phiwpn],
                       ])

# combine firm matrices in the following format:
# [Phiya]
# [Phiwpa]
# [Phirka]
dYFdZ = sp.sparse.bmat([[Phiya],
                       [Phirka],
                       [Phiwpa]])

assert dYFdU.shape == (3*T, 2*T)
assert dYFdZ.shape == (3*T, T)

TypeError: unsupported operand type(s) for /: 'int' and 'csr_matrix'

In [49]:
# firm block matrices: output
Phiyn = I
Phiym = Z
Phiyp = Z

# firm block matrices: labor demand
Phiwpm = Z
Phiwpp = Z
Phiwpn = Z

# household block matrix: labor supply
Phica = 1/gamma * Phiwpm
Phick = (1/A) * phi*I - (1/A)*Phiwpm
Phicn = Z


# there is a Phix() function that goes in to the bond makrket equations but I don't know how to calculate c for it 
#bond market 

Phiqm =nu*I*Phica - (nu-gamma)*I*Phigmx
Phiqn = nu*I*Phicn - (nu-gamma)*I*Phigmn

# combine firm matrices in the following format:
# [Phiyk Phiyn]
# [Phirkk Phirkn]
# [Phiwpk Phiwpn]
dYFdU = sp.sparse.bmat([[Phiym, Phiyn],
                        [Phirkk, Phirkn],
                        [Phiwpm, Phiwpn],
                       ])

# combine firm matrices in the following format:
# [Phiya]
# [Phiwpa]
# [Phirka]
dYFdZ = sp.sparse.bmat([[Phiya],
                       [Phirka],
                       [Phiwpa]])

assert dYFdU.shape == (3*T, 2*T)
assert dYFdZ.shape == (3*T, T)

TypeError: unsupported operand type(s) for /: 'int' and 'csr_matrix'

 household block

In [50]:
# household block matrix: labor supply
Phica = 1/gamma * Phiwpm
Phick = 1/A * phi*I - 1/A*Phiwpm
Phicn = Z

# household block matrix: capital accumulation equation
Phiia = Z
Phiik = (delta - 1) / delta * Im1 + 1 / delta * I
Phiin = Z

# combine household matrices in the following format:
# [Phick Phicn]
# [Phiik Phiin]
dYHdU = sp.sparse.bmat([[Phick, Phicn],
                       [Phiik, Phiin]])

# combine household matrices in the following format:
# [Phick Phicn]
# [Phiik Phiin]
dYHdZ = sp.sparse.bmat([[Phica],
                       [Phiia]])

assert dYHdU.shape == (2*T, 2*T)
assert dYHdZ.shape == (2*T, T)

TypeError: unsupported operand type(s) for /: 'int' and 'csr_matrix'

stack the household block on top of the firm block to get  ∂𝐘/∂𝐔  and  ∂𝐘/∂𝐙 .

In [51]:
# stack dYHdU on top of dYFdU to get dYdU
dYdU = sp.sparse.bmat([[dYHdU],
                      [dYFdU]])

# stack dYHdZ on top of dYFdZ to get dYdZ
dYdZ = sp.sparse.bmat([[dYHdZ],
                      [dYFdZ]])

assert dYdU.shape == (5*T, 2*T)
assert dYdZ.shape == (5*T, T)

NameError: name 'dYHdU' is not defined

solution

In [52]:
# compute dHdU using the chain rule dHdU = dHdY @ dYdU (@ is the python matrix multiplication operator)
dHdU = dHdY @ dYdU 

# compute dHdZ using the chain rule dHdZ = dHdY @ dYdZ (@ is the python matrix multiplication operator)
dHdZ = dHdY @ dYdZ

assert sp.sparse.issparse(dHdZ) == True
assert sp.sparse.issparse(dHdU) == True

assert dHdU.shape == (2*T, 2*T)
assert dHdZ.shape == (2*T, T)

# compute the Jacobian of the model
dUdZ = - sp.sparse.linalg.spsolve(dHdU, dHdZ)
dYdZ = dYdU @ dUdZ + dYdZ

dXdZ = sp.sparse.bmat([[dUdZ],
                      [dYdZ]])

assert dUdZ.shape == (2*T, T)
assert dYdZ.shape == (5*T, T)
assert dXdZ.shape == (7*T, T)

NameError: name 'dYdU' is not defined